In [1]:
import os
import sys
import time
import random
import pygame
import cv2
import math
import mediapipe as mp

# Constants and Settings
WINDOW_NAME = "Candy Collector"
GAME_TITLE = WINDOW_NAME

SCREEN_WIDTH, SCREEN_HEIGHT = 1200, 700
FPS = 90
DRAW_FPS = True

HAND_SIZE = 200
HAND_HITBOX_SIZE = (60, 80)
CANDY_SIZES = (50, 50)
CANDY_SIZE_RANDOMIZE = (1, 2)
BONUS_CANDY_SIZES = (50, 50)
BONUS_CANDY_SIZE_RANDOMIZE = (1.2, 1.5)

DRAW_HITBOX = False
ANIMATION_SPEED = 0.08
BUTTONS_SIZES = (240, 90)

GAME_DURATION = 60
CANDY_SPAWN_TIME = 2
CANDY_MOVE_SPEED = {"min": (2, 2), "max": (5, 5)}
BONUS_PENALTY = 1

GOAL_SCORE = 100
BONUS_TIME_GAIN = 5

COLORS = {
    "title": (38, 61, 39),
    "score": (38, 61, 39),
    "timer": (38, 61, 39),
    "buttons": {
        "default": (56, 67, 209),
        "second": (87, 99, 255),
        "text": (255, 255, 255),
        "shadow": (46, 54, 163)
    }
}

MUSIC_VOLUME = 0.1
SOUNDS_VOLUME = 0.5

pygame.font.init()
FONTS = {
    "small": pygame.font.Font(None, 40),
    "medium": pygame.font.Font(None, 72),
    "big": pygame.font.Font(None, 120)
}

# Tips Text
tips_text = [
    "Instructions:",
    "You have 60 seconds to finish this game.",
    "Collect candy by hovering your hand over them.",
    "To win you need to score 100 points before time runs out",
    "Use Golden candies to get extra 5 seconds on your time.",
    "Avoid Red candies or it will deduct 3 seconds from your time.",
    "GOOD LUCK!"
]

# Helper Functions
def load_image(img_path, size="default", convert="alpha", flip=False):
    if convert == "alpha":
        img = pygame.image.load(img_path).convert_alpha()
    else:
        img = pygame.image.load(img_path).convert()

    if flip:
        img = pygame.transform.flip(img, True, False)

    if size != "default":
        img = pygame.transform.smoothscale(img, size)

    return img

def draw_image(surface, img, pos, pos_mode="top_left"):
    if pos_mode == "center":
        pos = list(pos)
        pos[0] -= img.get_width() // 2
        pos[1] -= img.get_height() // 2
    surface.blit(img, pos)

def draw_text(surface, text, pos, color, font, pos_mode="top_left", shadow=False, shadow_color=(0, 0, 0), shadow_offset=2):
    label = font.render(text, 1, color)
    label_rect = label.get_rect()
    if pos_mode == "top_left":
        label_rect.x, label_rect.y = pos
    elif pos_mode == "center":
        label_rect.center = pos

    if shadow:
        label_shadow = font.render(text, 1, shadow_color)
        surface.blit(label_shadow, (label_rect.x - shadow_offset, label_rect.y + shadow_offset))

    surface.blit(label, label_rect)

def draw_tips(surface, tips_text, duration, width=700, height=400):
    pygame.time.set_timer(pygame.USEREVENT, duration * 1000)
    rect = pygame.Rect((SCREEN_WIDTH // 2 - width // 2, SCREEN_HEIGHT // 2 - height // 2), (width, height))
    pygame.draw.rect(surface, (255, 255, 255), rect)
    pygame.draw.rect(surface, (0, 0, 0), rect, 5)

    y_offset = rect.top + 20
    for line in tips_text:
        draw_text(surface, line, (SCREEN_WIDTH // 2, y_offset), (0, 0, 0), FONTS["small"], pos_mode="center")
        y_offset += 40

def button(surface, pos_y, text=None, click_sound=None):
    rect = pygame.Rect((SCREEN_WIDTH // 2 - BUTTONS_SIZES[0] // 2, pos_y), BUTTONS_SIZES)
    on_button = rect.collidepoint(pygame.mouse.get_pos())
    color = COLORS["buttons"]["second"] if on_button else COLORS["buttons"]["default"]

    pygame.draw.rect(surface, COLORS["buttons"]["shadow"], (rect.x - 6, rect.y - 6, rect.w, rect.h))
    pygame.draw.rect(surface, color, rect)
    if text:
        draw_text(surface, text, rect.center, COLORS["buttons"]["text"], FONTS["medium"], pos_mode="center", shadow=True, shadow_color=COLORS["buttons"]["shadow"])

    if on_button and pygame.mouse.get_pressed()[0]:
        if click_sound:
            click_sound.play()
        return True


# Classes
class Background:
    def __init__(self, image_path):
        self.image = load_image(image_path, size=(SCREEN_WIDTH, SCREEN_HEIGHT), convert="default")

    def draw(self, surface):
        draw_image(surface, self.image, (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2), pos_mode="center")

class Candy:
    def __init__(self):
        size = (int(CANDY_SIZES[0] * random.uniform(*CANDY_SIZE_RANDOMIZE)),
                int(CANDY_SIZES[1] * random.uniform(*CANDY_SIZE_RANDOMIZE)))
        self.rect = pygame.Rect(random.randint(0, SCREEN_WIDTH - size[0]),
                                -size[1],
                                size[0], size[1])
        self.image = load_image("Candy/candyy.png", size=size)
        self.velocity = [random.uniform(*CANDY_MOVE_SPEED["min"]),
                         random.uniform(*CANDY_MOVE_SPEED["max"])]
        self.is_collected = False

    def move(self):
        self.rect.y += self.velocity[1]
        if self.rect.top > SCREEN_HEIGHT:
            self.rect.y = -self.rect.height

    def draw(self, surface):
        if not self.is_collected:
            draw_image(surface, self.image, self.rect.topleft)

class BonusCandy(Candy):
    def __init__(self):
        size = (int(BONUS_CANDY_SIZES[0] * random.uniform(*BONUS_CANDY_SIZE_RANDOMIZE)),
                int(BONUS_CANDY_SIZES[1] * random.uniform(*BONUS_CANDY_SIZE_RANDOMIZE)))
        self.rect = pygame.Rect(random.randint(0, SCREEN_WIDTH - size[0]),
                                -size[1],
                                size[0], size[1])
        self.image = load_image("Candy/Gcandy.png", size=size)
        self.velocity = [random.uniform(*CANDY_MOVE_SPEED["min"]),
                         random.uniform(*CANDY_MOVE_SPEED["max"])]
        self.is_collected = False

class HandTracking:
    def __init__(self):
        self.hand_tracking = mp.solutions.hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
        self.hand_x = 0
        self.hand_y = 0
        self.results = None
        self.hand_closed = False

    def scan_hands(self, frame):
        image = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        self.results = self.hand_tracking.process(image)
        image.flags.writeable = True
        if self.results.multi_hand_landmarks:
            hand = self.results.multi_hand_landmarks[0]
            x_max = max([lm.x for lm in hand.landmark])
            x_min = min([lm.x for lm in hand.landmark])
            y_max = max([lm.y for lm in hand.landmark])
            y_min = min([lm.y for lm in hand.landmark])
            self.hand_closed = ((x_max - x_min) * (y_max - y_min)) < 0.1
            self.hand_x, self.hand_y = hand.landmark[9].x * SCREEN_WIDTH, hand.landmark[9].y * SCREEN_HEIGHT


class RedCandy(Candy):
    def __init__(self):
        size = (int(CANDY_SIZES[0] * random.uniform(*CANDY_SIZE_RANDOMIZE)),
                int(CANDY_SIZES[1] * random.uniform(*CANDY_SIZE_RANDOMIZE)))
        self.rect = pygame.Rect(random.randint(0, SCREEN_WIDTH - size[0]),
                                -size[1],
                                size[0], size[1])
        self.image = load_image("Candy/rcandy.png", size=size)
        self.velocity = [random.uniform(*CANDY_MOVE_SPEED["min"]),
                         random.uniform(*CANDY_MOVE_SPEED["max"])]
        self.is_collected = False

class Hand:
    def __init__(self):
        self.orig_image = load_image("Candy/candy_basket.png", size=(HAND_SIZE, HAND_SIZE))
        self.image = self.orig_image.copy()
        self.image_smaller = load_image("Candy/candy_basket.png", size=(HAND_SIZE - 50, HAND_SIZE - 50))
        self.rect = pygame.Rect(SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2, HAND_HITBOX_SIZE[0], HAND_HITBOX_SIZE[1])
        self.left_click = False

    def draw(self, surface):
        draw_image(surface, self.image, self.rect.center, pos_mode="center")
        if DRAW_HITBOX:
            pygame.draw.rect(surface, (200, 60, 0), self.rect)

    def update_position(self, x, y):
        self.rect.center = (x, y)
        self.image = self.image_smaller.copy() if self.left_click else self.orig_image.copy()

    def collect_candies(self, candies, bonus_candies, red_candies, game_duration):
        score = 0
        if self.left_click:
            for candy in candies[:]:
                if isinstance(candy, RedCandy) and self.rect.colliderect(candy.rect):
                    candies.remove(candy)
                    game_duration -= 3
                elif self.rect.colliderect(candy.rect):
                    candies.remove(candy)
                    score += 1
            for bonus_candy in bonus_candies[:]:
                if self.rect.colliderect(bonus_candy.rect):
                    bonus_candies.remove(bonus_candy)
                    game_duration += BONUS_TIME_GAIN
        return score, game_duration


pygame 2.5.2 (SDL 2.28.3, Python 3.11.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


ImportError: initialization failed

In [ ]:
class OpponentAI:
    def __init__(self):
        self.rect = pygame.Rect(SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2, HAND_HITBOX_SIZE[0], HAND_HITBOX_SIZE[1])
        self.velocity = [random.uniform(*CANDY_MOVE_SPEED["min"]), random.uniform(*CANDY_MOVE_SPEED["max"])]

    def move_towards_candy(self, candies):
        # Find the closest candy
        closest_candy = None
        min_distance = float('inf')
        for candy in candies:
            distance = math.sqrt((self.rect.centerx - candy.rect.centerx)**2 + (self.rect.centery - candy.rect.centery)**2)
            if distance < min_distance:
                closest_candy = candy
                min_distance = distance

        # Move towards the closest candy
        if closest_candy:
            dx = closest_candy.rect.centerx - self.rect.centerx
            dy = closest_candy.rect.centery - self.rect.centery
            distance = math.sqrt(dx**2 + dy**2)
            if distance > 0:
                self.velocity[0] = dx / distance * CANDY_MOVE_SPEED["max"][0]
                self.velocity[1] = dy / distance * CANDY_MOVE_SPEED["max"][1]

    def move(self):
        # Move opponent AI based on velocity
        self.rect.x += self.velocity[0]
        self.rect.y += self.velocity[1]

    def collect_candies(self, candies):
        # Check if opponent AI intersects with any candies
        for candy in candies[:]:
            if self.rect.colliderect(candy.rect):
                # Perform candy collection action (e.g., increase opponent's score)
                candies.remove(candy)



def main():
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    pygame.display.set_caption(GAME_TITLE)
    clock = pygame.time.Clock()

    bg = Background("Candy/cotton2.jpg")
    hand = Hand()
    hand_tracking = HandTracking()
    candies = []
    bonus_candies = []
    red_candies = []

    # Initialize opponent AI
    opponent_ai = OpponentAI()

    start_time = 0
    score = 0
    game_duration = GAME_DURATION
    game_over = False
    game_running = True
    game_active = False
    show_tips = False

    cap = cv2.VideoCapture(0)
    
    click_sound = pygame.mixer.Sound("Assets/Sounds/music.wav")

    while game_running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                game_running = False
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                game_running = False

        if not game_active:
            screen.fill((0, 0, 0))
            bg.draw(screen)
            draw_text(screen, "Candy Collector", (SCREEN_WIDTH // 2, 100), COLORS["title"], FONTS["big"], pos_mode="center")
            if button(screen, 300, "Start Game", click_sound):
                game_active = True
                start_time = time.time()
            if button(screen, 450, "Instruction!", click_sound):
                show_tips = True
                draw_tips(screen, tips_text, 3, width=900, height=600)
                show_tips = False
            if button(screen, 600, "Exit", click_sound):
                game_running = False
            pygame.display.flip()
            continue

        ret, frame = cap.read()
        if not ret:
            continue

        hand_tracking.scan_hands(frame)
        hand.update_position(hand_tracking.hand_x, hand_tracking.hand_y)
        hand.left_click = hand_tracking.hand_closed

        if len(candies) < 10:
            if random.random() < 0.05:
                if random.random() < 0.1:
                    candies.append(RedCandy())
                else:
                    candies.append(Candy())

        if len(bonus_candies) < 2:
            if random.random() < 0.01:
                bonus_candies.append(BonusCandy())

        for candy in candies:
            candy.move()
        
        for bonus_candy in bonus_candies:
            bonus_candy.move()

        collected_score, game_duration = hand.collect_candies(candies, bonus_candies, red_candies, game_duration)
        score += collected_score

        # Move and update opponent AI
        opponent_ai.move_towards_candy(candies)
        opponent_ai.move()
        opponent_ai.collect_candies(candies)

        screen.fill((0, 0, 0))
        bg.draw(screen)

        for candy in candies:
            candy.draw(screen)

        for bonus_candy in bonus_candies:
            bonus_candy.draw(screen)

        hand.draw(screen)

        if not game_over:
            elapsed_time = time.time() - start_time
            draw_text(screen, f"Score: {score}", (20, 20), COLORS["score"], FONTS["small"])
            draw_text(screen, f"Time: {int(game_duration - elapsed_time)}", (SCREEN_WIDTH - 20, 20), COLORS["timer"], FONTS["small"], pos_mode="top_right")

            if score >= GOAL_SCORE:
                display_message(screen, "You win!", (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2))
                game_over = True
            elif elapsed_time >= game_duration:
                display_message(screen, "Game Over", (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2))
                game_over = True

        pygame.display.flip()
        clock.tick(FPS)

        if game_over:
            pygame.time.delay(5000)
            game_running = False

    cap.release()
    pygame.quit()
    sys.exit()



In [ ]:
if __name__ == "__main__":
    main()

SystemExit: 

e:\china_2nd\Web dev\Projects\web_env\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Opponent 

In [5]:
import os
import sys
import time
import random
import pygame
import cv2
import mediapipe as mp
import math

# Constants and Settings
WINDOW_NAME = "Candy Collector"
GAME_TITLE = WINDOW_NAME

SCREEN_WIDTH, SCREEN_HEIGHT = 1200, 700
FPS = 90
DRAW_FPS = True

HAND_SIZE = 150
HAND_HITBOX_SIZE = (60, 80)
CANDY_SIZES = (50, 50)
CANDY_SIZE_RANDOMIZE = (1, 2)
BONUS_CANDY_SIZES = (50, 50)
BONUS_CANDY_SIZE_RANDOMIZE = (1.2, 1.5)
CANDY_GENERATION_PROBABILITY = 0.1

DRAW_HITBOX = False
ANIMATION_SPEED = 0.08
BUTTONS_SIZES = (240, 90)

GAME_DURATION = 60
CANDY_SPAWN_TIME = 2
CANDY_MOVE_SPEED = {"min": (4, 4), "max": (10, 10)}
BONUS_PENALTY = 1

GOAL_SCORE = 100
BONUS_TIME_GAIN = 5

COLORS = {
    "title": (38, 61, 39),
    "score": (38, 61, 39),
    "timer": (38, 61, 39),
    "buttons": {
        "default": (56, 67, 209),
        "second": (87, 99, 255),
        "text": (255, 255, 255),
        "shadow": (46, 54, 163)
    }
}

MUSIC_VOLUME = 0.1
SOUNDS_VOLUME = 0.5

pygame.font.init()
FONTS = {
    "small": pygame.font.Font(None, 40),
    "medium": pygame.font.Font(None, 72),
    "big": pygame.font.Font(None, 120)
}

# Tips Text
tips_text = [
    "Instructions:",
    " You have 60 seconds to finish this game.",
    " Collect candy by hovering your hand over them.",
    " To win you need to score 100 points before time runs out",
    " Use Golden candies to get extra 5 seconds on your time.",
    " Avoid Red candies or it will deduct 3 seconds from your time.",
    " You have an opponent who shares the candies so try to move faster!",
    " GOOD LUCK!"
]

# Helper Functions
def load_image(img_path, size="default", convert="alpha", flip=False):
    if convert == "alpha":
        img = pygame.image.load(img_path).convert_alpha()
    else:
        img = pygame.image.load(img_path).convert()

    if flip:
        img = pygame.transform.flip(img, True, False)

    if size != "default":
        img = pygame.transform.smoothscale(img, size)

    return img

def draw_image(surface, img, pos, pos_mode="top_left"):
    if pos_mode == "center":
        pos = list(pos)
        pos[0] -= img.get_width() // 2
        pos[1] -= img.get_height() // 2
    surface.blit(img, pos)

def draw_text(surface, text, pos, color, font, pos_mode="top_left", shadow=False, shadow_color=(0, 0, 0), shadow_offset=2):
    label = font.render(text, 1, color)
    label_rect = label.get_rect()
    if pos_mode == "top_left":
        label_rect.x, label_rect.y = pos
    elif pos_mode == "center":
        label_rect.center = pos

    if shadow:
        label_shadow = font.render(text, 1, shadow_color)
        surface.blit(label_shadow, (label_rect.x - shadow_offset, label_rect.y + shadow_offset))

    surface.blit(label, label_rect)

def draw_tips(surface, tips_text, duration, width=800, height=400):
    pygame.time.set_timer(pygame.USEREVENT, duration * 1000)
    rect = pygame.Rect((SCREEN_WIDTH // 2 - width // 2, SCREEN_HEIGHT // 2 - height // 2), (width, height))
    pygame.draw.rect(surface, (255, 255, 255), rect)
    pygame.draw.rect(surface, (0, 0, 0), rect, 5)

    y_offset = rect.top + 20
    for line in tips_text:
        draw_text(surface, line, (SCREEN_WIDTH // 2, y_offset), (0, 0, 0), FONTS["small"], pos_mode="center")
        y_offset += 40

def button(surface, pos_y, text=None, click_sound=None):
    rect = pygame.Rect((SCREEN_WIDTH // 2 - BUTTONS_SIZES[0] // 2, pos_y), BUTTONS_SIZES)
    on_button = rect.collidepoint(pygame.mouse.get_pos())
    color = COLORS["buttons"]["second"] if on_button else COLORS["buttons"]["default"]

    pygame.draw.rect(surface, COLORS["buttons"]["shadow"], (rect.x - 6, rect.y - 6, rect.w, rect.h))
    pygame.draw.rect(surface, color, rect)
    if text:
        draw_text(surface, text, rect.center, COLORS["buttons"]["text"], FONTS["medium"], pos_mode="center", shadow=True, shadow_color=COLORS["buttons"]["shadow"])

    if on_button and pygame.mouse.get_pressed()[0]:
        if click_sound:
            click_sound.play()
        return True


# Classes
class Background:
    def __init__(self, image_path):
        self.image = load_image(image_path, size=(SCREEN_WIDTH, SCREEN_HEIGHT), convert="default")

    def draw(self, surface):
        draw_image(surface, self.image, (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2), pos_mode="center")

class Candy:
    def __init__(self):
        size = (int(CANDY_SIZES[0] * random.uniform(*CANDY_SIZE_RANDOMIZE)),
                int(CANDY_SIZES[1] * random.uniform(*CANDY_SIZE_RANDOMIZE)))
        self.rect = pygame.Rect(random.randint(0, SCREEN_WIDTH - size[0]),
                                -size[1],
                                size[0], size[1])
        self.image = load_image("Candy/candyy.png", size=size)
        self.velocity = [random.uniform(*CANDY_MOVE_SPEED["min"]),
                         random.uniform(*CANDY_MOVE_SPEED["max"])]
        self.is_collected = False

    def move(self):
        self.rect.y += self.velocity[1]
        if self.rect.top > SCREEN_HEIGHT:
            self.rect.y = -self.rect.height

    def draw(self, surface):
        if not self.is_collected:
            draw_image(surface, self.image, self.rect.topleft)

class BonusCandy(Candy):
    def __init__(self):
        size = (int(BONUS_CANDY_SIZES[0] * random.uniform(*BONUS_CANDY_SIZE_RANDOMIZE)),
                int(BONUS_CANDY_SIZES[1] * random.uniform(*BONUS_CANDY_SIZE_RANDOMIZE)))
        self.rect = pygame.Rect(random.randint(0, SCREEN_WIDTH - size[0]),
                                -size[1],
                                size[0], size[1])
        self.image = load_image("Candy/Gcandy.png", size=size)
        self.velocity = [random.uniform(*CANDY_MOVE_SPEED["min"]),
                         random.uniform(*CANDY_MOVE_SPEED["max"])]
        self.is_collected = False

class HandTracking:
    def __init__(self):
        self.hand_tracking = mp.solutions.hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
        self.hand_x = 0
        self.hand_y = 0
        self.results = None
        self.hand_closed = False

    def scan_hands(self, frame):
        image = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        self.results = self.hand_tracking.process(image)
        image.flags.writeable = True
        if self.results.multi_hand_landmarks:
            hand = self.results.multi_hand_landmarks[0]
            x_max = max([lm.x for lm in hand.landmark])
            x_min = min([lm.x for lm in hand.landmark])
            y_max = max([lm.y for lm in hand.landmark])
            y_min = min([lm.y for lm in hand.landmark])
            self.hand_closed = ((x_max - x_min) * (y_max - y_min)) < 0.1
            self.hand_x, self.hand_y = hand.landmark[9].x * SCREEN_WIDTH, hand.landmark[9].y * SCREEN_HEIGHT


class RedCandy(Candy):
    def __init__(self):
        size = (int(CANDY_SIZES[0] * random.uniform(*CANDY_SIZE_RANDOMIZE)),
                int(CANDY_SIZES[1] * random.uniform(*CANDY_SIZE_RANDOMIZE)))
        self.rect = pygame.Rect(random.randint(0, SCREEN_WIDTH - size[0]),
                                -size[1],
                                size[0], size[1])
        self.image = load_image("Candy/rcandy.png", size=size)
        self.velocity = [random.uniform(*CANDY_MOVE_SPEED["min"]),
                         random.uniform(*CANDY_MOVE_SPEED["max"])]
        self.is_collected = False

class Hand:
    def __init__(self):
        self.orig_image = load_image("Candy/candy_basket.png", size=(HAND_SIZE, HAND_SIZE))
        self.image = self.orig_image.copy()
        self.image_smaller = load_image("Candy/candy_basket.png", size=(HAND_SIZE - 50, HAND_SIZE - 50))
        self.rect = pygame.Rect(SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2, HAND_HITBOX_SIZE[0], HAND_HITBOX_SIZE[1])
        self.left_click = False

    def draw(self, surface):
        draw_image(surface, self.image, self.rect.center, pos_mode="center")
        if DRAW_HITBOX:
            pygame.draw.rect(surface, (200, 60, 0), self.rect)

    def update_position(self, x, y):
        self.rect.center = (x, y)
        self.image = self.image_smaller.copy() if self.left_click else self.orig_image.copy()

    def collect_candies(self, candies, bonus_candies, red_candies, game_duration):
        score = 0
        if self.left_click:
            for candy in candies[:]:
                if isinstance(candy, RedCandy) and self.rect.colliderect(candy.rect):
                    candies.remove(candy)
                    game_duration -= 3
                elif self.rect.colliderect(candy.rect):
                    candies.remove(candy)
                    score += 1
            for bonus_candy in bonus_candies[:]:
                if self.rect.colliderect(bonus_candy.rect):
                    bonus_candies.remove(bonus_candy)
                    game_duration += BONUS_TIME_GAIN
        return score, game_duration


class OpponentAI:
    def __init__(self):
        self.rect = pygame.Rect(SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2, HAND_HITBOX_SIZE[0], HAND_HITBOX_SIZE[1])
        self.velocity = [0, 0]
        self.target_candy = None  # Initialize target candy to None

    def choose_target_candy(self, candies, bonus_candies, red_candies):
        closest_candy_dist = float('inf')
        for candy in candies + bonus_candies:
            if not candy.is_collected:
                if isinstance(candy, RedCandy):  # Skip red candies
                    continue
                dist_to_candy = math.sqrt((candy.rect.x - self.rect.x) ** 2 + (candy.rect.y - self.rect.y) ** 2)
                if dist_to_candy < closest_candy_dist:
                    closest_candy_dist = dist_to_candy
                    self.target_candy = candy

    def move_towards_target(self):
        if self.target_candy:
            delta_x = self.target_candy.rect.x - self.rect.x
            delta_y = self.target_candy.rect.y - self.rect.y
            distance = math.sqrt(delta_x ** 2 + delta_y ** 2)
            if distance > 0:
                self.velocity = [delta_x / distance * 5, delta_y / distance * 5]
        else:
            self.velocity = [0, 0]

    def move(self):
        self.rect.x += self.velocity[0]
        self.rect.y += self.velocity[1]

    def collect_candies(self, candies, bonus_candies):
        for candy in candies[:]:
            if self.rect.colliderect(candy.rect):
                candies.remove(candy)
                self.target_candy = None

        for bonus_candy in bonus_candies[:]:
            if self.rect.colliderect(bonus_candy.rect):
                bonus_candies.remove(bonus_candy)
                self.target_candy = None

    def draw(self, surface):
        opponent_img = load_image("Candy/opponent2.png", size=(HAND_HITBOX_SIZE[0], HAND_HITBOX_SIZE[1]))
        draw_image(surface, opponent_img, self.rect.topleft)


# Functions

def display_message(surface, message, position):
    # Function to display a message on the screen
    font = pygame.font.Font(None, 72)
    text = font.render(message, True, (255, 0, 0))
    rect = text.get_rect(center=position)
    surface.blit(text, rect)

def main():
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    pygame.display.set_caption(GAME_TITLE)
    clock = pygame.time.Clock()

    bg = Background("Candy/cotton2.jpg")
    hand = Hand()
    hand_tracking = HandTracking()
    candies = []
    bonus_candies = []
    red_candies = []
    opponent = OpponentAI()  # Move this initialization here

    start_time = 0
    score = 0
    game_duration = GAME_DURATION
    game_over = False
    game_running = True
    game_active = False
    show_tips = False

    cap = cv2.VideoCapture(0)
    
    click_sound = pygame.mixer.Sound("Assets/Sounds/music.wav")

    while game_running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                game_running = False
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                game_running = False

        if not game_active:
            screen.fill((0, 0, 0))
            bg.draw(screen)
            draw_text(screen, "Candy Collector", (SCREEN_WIDTH // 2, 100), COLORS["title"], FONTS["big"], pos_mode="center")
            if button(screen, 300, "Start Game", click_sound):
                game_active = True
                start_time = time.time()
            if button(screen, 450, "Tips!", click_sound):
                show_tips = True
                draw_tips(screen, tips_text, 3, width=1000, height=600)
                show_tips = False
            if button(screen, 600, "Exit", click_sound):
                game_running = False
            pygame.display.flip()
            continue

        ret, frame = cap.read()
        if not ret:
            continue

        hand_tracking.scan_hands(frame)
        hand.update_position(hand_tracking.hand_x, hand_tracking.hand_y)
        hand.left_click = hand_tracking.hand_closed

        if len(candies) < 10:
            if random.random() < 0.05:
                if random.random() < 0.1:
                    candies.append(RedCandy())
                else:
                    candies.append(Candy())

        if len(bonus_candies) < 2:
            if random.random() < 0.01:
                bonus_candies.append(BonusCandy())

        for candy in candies:
            candy.move()
        
        for bonus_candy in bonus_candies:
            bonus_candy.move()

        collected_score, game_duration = hand.collect_candies(candies, bonus_candies, red_candies, game_duration)
        score += collected_score
        
        opponent.choose_target_candy(candies,bonus_candies, red_candies)  # Choose target candy
        opponent.move_towards_target()  # Move towards target candy
        opponent.move()  # Move opponent
        opponent.collect_candies(candies,bonus_candies)  # Collect candies


        screen.fill((0, 0, 0))
        bg.draw(screen)

        for candy in candies:
            candy.draw(screen)

        for bonus_candy in bonus_candies:
            bonus_candy.draw(screen)

        hand.draw(screen)
        opponent.draw(screen)

        if not game_over:
            elapsed_time = time.time() - start_time
            draw_text(screen, f"Score: {score}", (20, 20), COLORS["score"], FONTS["small"])
            draw_text(screen, f"Time: {int(game_duration - elapsed_time)}", (SCREEN_WIDTH - 20, 20), COLORS["timer"], FONTS["small"], pos_mode="top_right")

            if score >= GOAL_SCORE:
                display_message(screen, "You win!", (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2))
                game_over = True
            elif elapsed_time >= game_duration:
                display_message(screen, "Game Over", (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2))
                game_over = True

        pygame.display.flip()
        clock.tick(FPS)

        if game_over:
            pygame.time.delay(5000)
            game_running = False
    
# Update candy generation logic
    if len(candies) < 20 and random.random() < CANDY_GENERATION_PROBABILITY:
        if random.random() < 0.2:
            if random.random() < 0.1:
                candies.append(RedCandy())
            else:
                candies.append(Candy())

    # Spread candies across the screen
    for candy in candies:
        candy.move()
        if candy.rect.left < 0:
            candy.rect.right = SCREEN_WIDTH
        elif candy.rect.right > SCREEN_WIDTH:
            candy.rect.left = 0
        if candy.rect.top < 0:
            candy.rect.bottom = SCREEN_HEIGHT
        elif candy.rect.bottom > SCREEN_HEIGHT:
            candy.rect.top = 0

    cap.release()
    pygame.quit()
    sys.exit()


if __name__ == "__main__":
    main()


SystemExit: 